In [1]:
!pip install networkx tqdm
!pip install openai tiktoken --quiet

!pip -q install pyvis
!pip -q install -U jinja2 markupsafe pyvis


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.8 MB/s eta 0:00:00


In [2]:
PROJECT_PATH = "/content/drive/MyDrive/ProjectsforFinalproject/flaskbb"


In [3]:
# پیدا کردن فایل های پایتون و ایگنور فایل های غیر ضروری
import os
from tqdm import tqdm
def find_python_files(path):
    py_files = []
    for root, dirs, files in os.walk(path):
        if any(x in root for x in ["venv", "env", "__pycache__", "migrations", "tests"]):
            continue

        for f in files:
            if f.endswith(".py"):
                py_files.append(os.path.join(root, f))
    return py_files

python_files = find_python_files(PROJECT_PATH)
len(python_files), python_files[:5]

(90,
 ['/content/drive/MyDrive/ProjectsforFinalproject/flaskbb/celery_worker.py',
  '/content/drive/MyDrive/ProjectsforFinalproject/flaskbb/wsgi.py',
  '/content/drive/MyDrive/ProjectsforFinalproject/flaskbb/setup.py',
  '/content/drive/MyDrive/ProjectsforFinalproject/flaskbb/docs/conf.py',
  '/content/drive/MyDrive/ProjectsforFinalproject/flaskbb/flaskbb/extensions.py'])

In [4]:
#  استخراح و ساخت AST
import ast
import json

nodes = []
edges = []


node_id_counter = 0

# def new_node(n_type, name, file_path, extra=None):
def new_node(n_type, name, file_path):
    """ایجاد یک نود جدید"""
    global node_id_counter
    nid = node_id_counter
    node_id_counter += 1

    nodes.append({
        "id": nid,
        "type": n_type,
        "name": name,
        "file": file_path,
        # "extra": extra or {}
    })
    return nid


# نودهای فایل
file_nodes = {}   # map: file_path → node_id
for f in python_files:
    file_nodes[f] = new_node("file", os.path.basename(f), f)

# استخراج Class و Function

function_nodes = {}   # map: (file, func_name) → node_id
class_nodes = {}      # map: (file, class_name) → node_id

for f in tqdm(python_files):
    try:
        src = open(f, "r", encoding="utf-8").read()
        tree = ast.parse(src)
    except:
        continue

    for node in ast.walk(tree):
        if isinstance(node, ast.ClassDef):
            cid = new_node("class", node.name, f)
            class_nodes[(f, node.name)] = cid

            # یال File → Class
            edges.append({
                "src": file_nodes[f],
                "dst": cid,
                "type": "contains"
            })

        elif isinstance(node, ast.FunctionDef):
            fid = new_node("function", node.name, f)
            function_nodes[(f, node.name)] = fid

            # یال File → Function
            edges.append({
                "src": file_nodes[f],
                "dst": fid,
                "type": "contains"
            })

100%|██████████| 90/90 [00:37<00:00,  2.42it/s]


In [5]:
# @title
# #Filling Extra node value commented for now
# import os
# import ast
# from tqdm import tqdm

# nodes = []
# edges = []
# node_id_counter = 0

# def new_node(n_type, name, file_path, extra=None):
#     """ایجاد یک نود جدید"""
#     global node_id_counter
#     nid = node_id_counter
#     node_id_counter += 1

#     nodes.append({
#         "id": nid,
#         "type": n_type,
#         "name": name,
#         "file": file_path,
#         "extra": extra or {}
#     })
#     return nid


# # -----------------------------
# # 1) ساخت نودهای فایل
# # -----------------------------
# file_nodes = {}  # file_path -> node_id
# for f in python_files:
#     file_nodes[f] = new_node(
#         "file",
#         os.path.basename(f),
#         f,
#         extra={"file_path": f}
#     )


# # -----------------------------
# # 2) Visitor برای استخراج کلاس/تابع با context
# # -----------------------------
# class CodeEntityVisitor(ast.NodeVisitor):
#     def __init__(self, file_path):
#         self.file_path = file_path
#         self.class_stack = []   # برای تشخیص اینکه داخل کلاس هستیم یا نه

#     def _get_lines(self, node):
#         start = getattr(node, "lineno", None)
#         end = getattr(node, "end_lineno", None)
#         return start, end

#     def _get_docstring(self, node):
#         try:
#             return ast.get_docstring(node)
#         except Exception:
#             return None

#     def _get_decorators(self, node):
#         # decorator ها را به شکل متن ساده نگه می‌داریم
#         decs = []
#         for d in getattr(node, "decorator_list", []) or []:
#             try:
#                 decs.append(ast.unparse(d))  # py>=3.9
#             except Exception:
#                 decs.append(d.__class__.__name__)
#         return decs

#     def _get_args_brief(self, func_node):
#         # خلاصه امضای تابع: فقط نام آرگومان‌ها + وجود vararg/kwarg
#         args = []
#         try:
#             args = [a.arg for a in func_node.args.args]
#         except Exception:
#             args = []

#         return {
#             "args": args,
#             "vararg": getattr(func_node.args, "vararg", None).arg if getattr(func_node.args, "vararg", None) else None,
#             "kwarg": getattr(func_node.args, "kwarg", None).arg if getattr(func_node.args, "kwarg", None) else None,
#         }

#     def _qualify(self, name):
#         # qualified_name حداقلی: اگر داخل کلاس باشیم: Class.method
#         if self.class_stack:
#             return ".".join(self.class_stack + [name])
#         return name

#     def visit_ClassDef(self, node: ast.ClassDef):
#         start, end = self._get_lines(node)
#         qname = self._qualify(node.name)  # اگر nested class باشد هم پوشش می‌دهد

#         extra = {
#             "qualified_name": qname,
#             "start_line": start,
#             "end_line": end,
#             "docstring": self._get_docstring(node),
#             "decorators": self._get_decorators(node),
#             "parent_class": self.class_stack[-1] if self.class_stack else None,
#         }

#         cid = new_node("class", node.name, self.file_path, extra=extra)

#         # File -> Class (contains)
#         edges.append({"src": file_nodes[self.file_path], "dst": cid, "type": "contains"})

#         # اگر کلاس داخل کلاس باشد: ParentClass -> Class (contains)
#         if self.class_stack:
#             parent_qname = ".".join(self.class_stack)
#             parent_cid = class_nodes.get((self.file_path, parent_qname))
#             if parent_cid is not None:
#                 edges.append({"src": parent_cid, "dst": cid, "type": "contains"})

#         # ثبت کلاس با کلید دقیق (file, qualified_name)
#         class_nodes[(self.file_path, qname)] = cid

#         # ورود به کلاس و ادامه پیمایش برای متدها/کلاس‌های داخلی
#         self.class_stack.append(node.name)
#         self.generic_visit(node)
#         self.class_stack.pop()

#     def visit_FunctionDef(self, node: ast.FunctionDef):
#         self._handle_function_like(node, is_async=False)

#     def visit_AsyncFunctionDef(self, node: ast.AsyncFunctionDef):
#         self._handle_function_like(node, is_async=True)

#     def _handle_function_like(self, node, is_async: bool):
#         start, end = self._get_lines(node)
#         qname = self._qualify(node.name)

#         is_method = bool(self.class_stack)
#         parent_class = self.class_stack[-1] if self.class_stack else None

#         extra = {
#             "qualified_name": qname,
#             "start_line": start,
#             "end_line": end,
#             "docstring": self._get_docstring(node),
#             "decorators": self._get_decorators(node),
#             "is_async": is_async,
#             "is_method": is_method,
#             "parent_class": parent_class,
#             **self._get_args_brief(node),
#         }

#         fid = new_node("function", node.name, self.file_path, extra=extra)

#         # اگر متد است: Class -> Function (contains)
#         if is_method:
#             parent_qname = ".".join(self.class_stack)
#             parent_cid = class_nodes.get((self.file_path, parent_qname))
#             if parent_cid is not None:
#                 edges.append({"src": parent_cid, "dst": fid, "type": "contains"})
#             else:
#                 # fallback اگر به هر دلیل کلاس parent ثبت نشده بود
#                 edges.append({"src": file_nodes[self.file_path], "dst": fid, "type": "contains"})
#         else:
#             # تابع سطح فایل: File -> Function
#             edges.append({"src": file_nodes[self.file_path], "dst": fid, "type": "contains"})

#         # ثبت تابع با کلید دقیق (file, qualified_name)
#         function_nodes[(self.file_path, qname)] = fid

#         # ادامه پیمایش (اگر تابع داخل تابع داشته باشیم)
#         self.generic_visit(node)


# # -----------------------------
# # 3) اجرای استخراج روی همه فایل‌ها
# # -----------------------------
# function_nodes = {}  # (file, qualified_name) -> node_id
# class_nodes = {}     # (file, qualified_name) -> node_id

# for f in tqdm(python_files):
#     try:
#         with open(f, "r", encoding="utf-8", errors="ignore") as fp:
#             src = fp.read()
#         tree = ast.parse(src, filename=f)
#     except Exception:
#         continue

#     visitor = CodeEntityVisitor(f)
#     visitor.visit(tree)


In [6]:
#  ساخت یال Import
def extract_imports(tree, file_path):
    imports = []
    for node in ast.walk(tree):
        if isinstance(node, ast.Import):
            for alias in node.names:
                imports.append(alias.name)
        elif isinstance(node, ast.ImportFrom):
            mod = node.module if node.module else ""
            imports.append(mod)
    return imports


for f in tqdm(python_files):
    try:
        src = open(f, "r", encoding="utf-8").read()
        tree = ast.parse(src)
    except:
        continue

    imports = extract_imports(tree, f)

    for im in imports:
        edges.append({
            "src": file_nodes[f],
            "dst": new_node("module", im, f),
            "type": "import"
        })

100%|██████████| 90/90 [00:00<00:00, 177.70it/s]


In [7]:
#  ساخت یال CALL

def get_func_name(call_node):
    """استخراج نام تابع"""
    if isinstance(call_node.func, ast.Name):
        return call_node.func.id
    elif isinstance(call_node.func, ast.Attribute):
        return call_node.func.attr
    return None


for f in tqdm(python_files):
    try:
        src = open(f, "r", encoding="utf-8").read()
        tree = ast.parse(src)
    except:
        continue

    # پیدا کردن تابع فعلی برای ساخت یال call
    current_func_node_id = None

    for node in ast.walk(tree):

        # موقع ورود به تابع
        if isinstance(node, ast.FunctionDef):
            current_func_node_id = function_nodes.get((f, node.name), None)

        # فراخوانی تابع
        if isinstance(node, ast.Call) and current_func_node_id is not None:
            callee = get_func_name(node)

            # پیدا کردن callee در پروژه (در فایل‌های دیگر)
            callee_node_id = None
            for (file2, name2), nid in function_nodes.items():
                if name2 == callee:
                    callee_node_id = nid
                    break

            if callee_node_id:
                edges.append({
                    "src": current_func_node_id,
                    "dst": callee_node_id,
                    "type": "call"
                })

100%|██████████| 90/90 [00:00<00:00, 103.61it/s]


In [8]:
# ذخیره خروجی به صورت JSON
with open("/content/nodes.json", "w", encoding="utf-8") as f:
    json.dump(nodes, f, indent=2)

with open("/content/edges.json", "w", encoding="utf-8") as f:
    json.dump(edges, f, indent=2)

print("Done! فایل‌ها تولید شدند: nodes.json و edges.json")

Done! فایل‌ها تولید شدند: nodes.json و edges.json


In [9]:
import json

nodes = json.load(open("/content/nodes.json"))
node_types = {n["type"] for n in nodes}
print(node_types)


files = [n for n in nodes if n["type"] == "file"]
classes = [n for n in nodes if n["type"] == "class"]
funcs = [n for n in nodes if n["type"] == "function"]
Module = [n for n in nodes if n["type"] == "module"]

print(len(files), len(classes), len(funcs), len(Module))

{'class', 'function', 'file', 'module'}
90 237 761 526


In [10]:
!pip install pyvis
!pip -q install --upgrade pyvis jinja2


from pyvis.network import Network
import json

nodes = json.load(open("/content/nodes.json", "r", encoding="utf-8"))
edges = json.load(open("/content/edges.json", "r", encoding="utf-8"))

net = Network(height="800px", width="100%", bgcolor="white", directed=True, notebook=True)

for n in nodes:
    net.add_node(n["id"], label=n.get("name",""), title=n.get("type",""), group=n.get("type","unknown"))

for e in edges:
    net.add_edge(e["src"], e["dst"], title=e.get("type",""), label=e.get("type",""))

net.write_html("/content/code_graph.html")
print("Saved: /content/code_graph.html")



Saved: /content/code_graph.html


ChatGpt Request

In [11]:
!pip install tiktoken openai tqdm


In [12]:
import json, os, ast
from tqdm import tqdm

nodes = json.load(open("/content/nodes.json"))

def load_code(file_path):
    try:
        return open(file_path, "r", encoding="utf-8").read()
    except:
        return ""


In [13]:
import ast
import os

def extract_node_source(file_path, node_name, node_type):
    """
    از فایل، فقط بدنه گره (class/function) را استخراج می‌کند.
    اگر پیدا نشود، کل فایل را برمی‌گرداند.
    """

    try:
        src = open(file_path, "r", encoding="utf-8").read()
    except:
        return ""

    try:
        tree = ast.parse(src)
    except:
        return src  # fallback

    for node in ast.walk(tree):
        # استخراج بدنه تابع
        if node_type == "function" and isinstance(node, ast.FunctionDef):
            if node.name == node_name:
                start = node.lineno - 1
                end = node.end_lineno
                return "\n".join(src.splitlines()[start:end])

        # استخراج بدنه کلاس
        if node_type == "class" and isinstance(node, ast.ClassDef):
            if node.name == node_name:
                start = node.lineno - 1
                end = node.end_lineno
                return "\n".join(src.splitlines()[start:end])

    # اگر هیچ‌کدام پیدا نشد → کل فایل را برگردان
    return src


def build_embedding_text(node):
    """
    متن کامل و بهینه برای مدل embedding را می‌سازد.
    ساختار:
    - نوع گره
    - نام
    - مسیر فایل
    - بخشی از کد
    """

    file_path = node["file"]
    name = node["name"]
    node_type = node["type"]

    # بهترین حالت: فقط بدنه گره
    code_snippet = extract_node_source(file_path, name, node_type)

    text = f"""
### NODE INFORMATION ###
Type: {node_type.upper()}
Name: {name}
File: {file_path}

### CONTEXT ###
This node comes from a Python project.
Provide a semantic representation of its behavior and responsibilities.

### SOURCE CODE ###
{code_snippet}

### END ###
    """

    return text.strip()


In [14]:
# GPT Embeddings Function

import time
from openai import OpenAI

METIS_KEY = "tpsg-sH8SGowLyM3C6xDkKCpbuNlWhkqW8MB"

client = OpenAI(
    api_key = METIS_KEY,
    base_url = "https://api.tapsage.com/openai/v1"
)
def get_embedding(text, max_len=5000, retry_count=5):
    """
    گرفتن embedding از مدل GPT با مدیریت خطا و retry.
    """

    # کوتاه‌سازی متن برای جلوگیری از خطا
    text = text[:max_len]

    for attempt in range(retry_count):
        try:
            response = client.embeddings.create(
                model="text-embedding-3-large",
                input=text
            )
            return response.data[0].embedding

        except Exception as e:
            print(f"⚠️ Error: {e}")
            print(f"Retrying ({attempt+1}/{retry_count})...")
            time.sleep(2)

    print("❌ Failed to get embedding after retries.")
    return None


In [ ]:
import json
from collections import defaultdict
from tqdm import tqdm

nodes = json.load(open("/content/nodes.json", "r", encoding="utf-8"))
edges = json.load(open("/content/edges.json", "r", encoding="utf-8"))

node_by_id = {n["id"]: n for n in nodes}

adj = defaultdict(list)
for e in edges:
    s, d, t = e["src"], e["dst"], e["type"]
    adj[s].append((t, d, "out"))
    adj[d].append((t, s, "in"))

def build_embedding_text_with_edges(node, max_neighbors=25):
    """
    Build embedding text using:
    - node metadata
    - code snippet (from your existing build_embedding_text(node))
    - small local graph context (neighbors + edge types)
    """
    base_text = build_embedding_text(node)

    nid = node["id"]
    neighbors = adj.get(nid, [])[:max_neighbors]

    edge_lines = []
    for etype, nb_id, direction in neighbors:
        nb = node_by_id.get(nb_id)
        if nb is None:
            continue
        edge_lines.append(
            f"- ({direction}) {etype} -> [{nb['type']}] {nb['name']}"
        )

    edge_context = "\n".join(edge_lines) if edge_lines else "None"

    return f"""{base_text}

### LOCAL GRAPH CONTEXT (1-hop) ###
Node id: {nid}
Neighbors (limited to {max_neighbors}):
{edge_context}
""".strip()

node_embeddings = {}

for node in tqdm(nodes, desc="Embedding nodes+edges"):
    text = build_embedding_text_with_edges(node, max_neighbors=25)
    emb = get_embedding(text)

    if emb is None:
        print(f"Skipped node {node['id']} due to embedding failure.")
        continue

    node_embeddings[str(node["id"])] = emb

# Save
out_path = "/content/node_embeddings.json"
with open(out_path, "w", encoding="utf-8") as f:
    json.dump(node_embeddings, f)

print("DONE! Embeddings saved to:", out_path)


Embedding nodes+edges:  20%|█▉        | 318/1614 [19:01<59:12,  2.74s/it]  

In [ ]:
import json

path = "/content/node_embeddings.json"
emb = json.load(open(path, "r", encoding="utf-8"))

print("Total nodes embedded:", len(emb))
print("First 10 node ids:", list(emb.keys())[:10])

sample_id = list(emb.keys())[10]
print("Sample node id:", sample_id)
print("Vector length:", len(emb[sample_id]))
print("First 10 dims:", emb[sample_id][:10])


In [ ]:
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
emb = json.load(open("/content/node_embeddings.json"))
nodes = json.load(open("/content/nodes.json", "r", encoding="utf-8"))

lengths = set(len(v) for v in emb.values())
print(lengths)

emb_list = list(emb.values())
unique_count = len({tuple(v) for v in emb_list})
print("Unique vectors:", unique_count, "/", len(emb_list))


sample = list(emb.values())[:50]
sim = cosine_similarity(sample)

print("Mean similarity:", sim.mean())
print("Max similarity:", sim.max())
print("Min similarity:", sim.min())
bad_nodes = []

for node in nodes:
    text = build_embedding_text(node)
    if len(text.strip()) < 20:
        bad_nodes.append(node["id"])

len(bad_nodes), bad_nodes[:10]


print("Bad nodes number", len(bad_nodes))
target_id = "39" #Random Node For test

target = np.array(emb[target_id]).reshape(1, -1)
all_vecs = np.array(list(emb.values()))
node_ids = list(emb.keys())

sims = cosine_similarity(target, all_vecs)[0]
top = np.argsort(sims)[::-1][:10]

for i in top:
    print(node_ids[i], sims[i])


In [ ]:
k_values = range(4, 30)

results = []

for k in range(8, 30):
    # --- clustering ---
    labels = cop_kmeans(X_gnn, k, must, cannot, max_iter=100, seed=42)
    labels = fill_unassigned_by_nearest_center(X_gnn, labels, k)

    # --- evaluation on HELD-OUT edges ---
    coh, cou, used, intra, inter = cohesion_coupling_from_edges(
        labels, test_edges, global_to_local_func
    )

    score = coh - cou   # λ = 1

    cluster_sizes = [np.sum(labels == i) for i in range(k)]

    results.append({
        "k": k,
        "cohesion": coh,
        "coupling": cou,
        "score": score,
        "min_size": min(cluster_sizes),
        "max_size": max(cluster_sizes),
    })
    best_score = max(r["score"] for r in results)

candidates = [
    r for r in results
    if r["score"] >= 0.95 * best_score
    and r["min_size"] >= 5
    and r["max_size"] <= 0.5 * len(function_nodes)
]

best_k = min(r["k"] for r in candidates)
print("Best k:", best_k)


GNN Training

In [ ]:
!pip install torch torch-geometric scikit-learn -q

import json
import torch
from torch_geometric.data import HeteroData
from collections import defaultdict

NODES_PATH = "/content/nodes.json"
EDGES_PATH = "/content/edges.json"
EMB_PATH   = "/content/node_embeddings.json"

nodes = json.load(open(NODES_PATH, "r", encoding="utf-8"))
edges = json.load(open(EDGES_PATH, "r", encoding="utf-8"))
emb  = json.load(open(EMB_PATH, "r", encoding="utf-8"))

# node id -> type
id_to_type = {n["id"]: n["type"] for n in nodes}

# group ids per type
type_to_ids = defaultdict(list)
for n in nodes:
    type_to_ids[n["type"]].append(n["id"])

# local index per type
local_index = {t: {nid:i for i, nid in enumerate(ids)} for t, ids in type_to_ids.items()}

# embedding dim
first_key = next(iter(emb.keys()))
emb_dim = len(emb[first_key])

data = HeteroData()

# node features per type
for t, ids in type_to_ids.items():
    x = torch.zeros((len(ids), emb_dim), dtype=torch.float32)
    for nid in ids:
        v = emb.get(str(nid))
        if v is not None:
            x[local_index[t][nid]] = torch.tensor(v, dtype=torch.float32)
    data[t].x = x

# edges per relation
edge_dict = defaultdict(list)

for e in edges:
    et = e["type"]
    s, d = e["src"], e["dst"]
    if s not in id_to_type or d not in id_to_type:
        continue
    st, dt = id_to_type[s], id_to_type[d]

    if et == "contains":
        if st == "file" and dt in ("class", "function"):
            rel = ("file", "contains", dt)
        elif st == "class" and dt == "function":
            rel = ("class", "contains", "function")
        else:
            continue
    elif et == "import":
        if st == "file" and dt == "module":
            rel = ("file", "imports", "module")
        else:
            continue
    elif et == "call":
        if st == "function" and dt == "function":
            rel = ("function", "calls", "function")
        else:
            continue
    else:
        continue

    edge_dict[rel].append((local_index[st][s], local_index[dt][d]))

for rel, pairs in edge_dict.items():
    src, dst = zip(*pairs) if pairs else ([], [])
    data[rel].edge_index = torch.tensor([src, dst], dtype=torch.long)

print(data)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import HeteroConv, SAGEConv

class HeteroRGCN(nn.Module):
    def __init__(self, metadata, in_dim, hidden_dim, out_dim):
        super().__init__()
        self.node_lin = nn.ModuleDict({t: nn.Linear(in_dim, hidden_dim) for t in metadata[0]})

        self.conv1 = HeteroConv({et: SAGEConv((-1, -1), hidden_dim) for et in metadata[1]})
        self.conv2 = HeteroConv({et: SAGEConv((-1, -1), out_dim)    for et in metadata[1]})

    def forward(self, x_dict, edge_index_dict):
        x0 = {t: self.node_lin[t](x) for t, x in x_dict.items()}

        x1 = self.conv1(x0, edge_index_dict)
        for t in x0:
            x1[t] = x0[t] if x1.get(t) is None else F.relu(x1[t])

        x2 = self.conv2(x1, edge_index_dict)
        for t in x1:
            x2[t] = x1[t] if x2.get(t) is None else x2[t]

        return x2

In [ ]:
import torch
from torch_geometric.utils import negative_sampling
from sklearn.metrics import roc_auc_score
import numpy as np
from copy import deepcopy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
data = data.to(device)

edge_type = ("function", "calls", "function")
edge_index = data[edge_type].edge_index
m = edge_index.size(1)

perm = torch.randperm(m, device=device)
train_m = int(0.9 * m)
train_ei = edge_index[:, perm[:train_m]]
val_ei   = edge_index[:, perm[train_m:]]

def edge_score(z_src, z_dst, ei):
    return (z_src[ei[0]] * z_dst[ei[1]]).sum(dim=-1)

bce = nn.BCEWithLogitsLoss()

def loss_fn(model, pos_ei):
    model.train()
    z = model(data.x_dict, data.edge_index_dict)
    zf = z["function"]

    pos = edge_score(zf, zf, pos_ei)
    pos_y = torch.ones(pos.size(0), device=device)

    neg_ei = negative_sampling(pos_ei, num_nodes=zf.size(0), num_neg_samples=pos_ei.size(1))
    neg = edge_score(zf, zf, neg_ei)
    neg_y = torch.zeros(neg.size(0), device=device)

    s = torch.cat([pos, neg])
    y = torch.cat([pos_y, neg_y])
    return bce(s, y)

@torch.no_grad()
def eval_auc(model, pos_ei, num_neg=5000):
    model.eval()
    z = model(data.x_dict, data.edge_index_dict)
    zf = z["function"]

    pos = edge_score(zf, zf, pos_ei)
    pos_y = torch.ones(pos.size(0))

    neg_ei = negative_sampling(pos_ei, num_nodes=zf.size(0), num_neg_samples=min(num_neg, pos_ei.size(1)))
    neg = edge_score(zf, zf, neg_ei)
    neg_y = torch.zeros(neg.size(0))

    y = torch.cat([pos_y, neg_y]).cpu().numpy()
    s = torch.cat([pos, neg]).cpu().numpy()
    return roc_auc_score(y, s)

in_dim = data["function"].x.size(1)
model = HeteroRGCN(data.metadata(), in_dim=in_dim, hidden_dim=256, out_dim=128).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

best_auc = 0.0
best_state = None

for epoch in range(1, 101):
    loss = loss_fn(model, train_ei)
    opt.zero_grad()
    loss.backward()
    opt.step()

    if epoch == 1 or epoch % 10 == 0:
        auc = eval_auc(model, val_ei)
        print(f"Epoch {epoch:03d} | Loss {loss.item():.4f} | Val AUC {auc:.4f}")
        if auc > best_auc:
            best_auc = auc
            best_state = deepcopy(model.state_dict())

if best_state:
    model.load_state_dict(best_state)
print("Best Val AUC:", best_auc)


In [ ]:
@torch.no_grad()
def export_embeddings(model, out_path="/content/gnn_embeddings.json"):
    model.eval()
    z = model(data.x_dict, data.edge_index_dict)
    z_func = z["function"].detach().cpu().numpy()
    out = {str(i): z_func[i].tolist() for i in range(z_func.shape[0])}
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(out, f)
    print("Saved:", out_path, "shape:", z_func.shape)

export_embeddings(model)


In [ ]:
import torch
from torch_geometric.utils import negative_sampling

@torch.no_grad()
# def eval_hits_mrr(model, data, pos_ei, K=15, num_neg_per_pos=50):
def eval_hits_mrr(model, data, pos_ei, K=10, num_neg_per_pos=50):
    model.eval()
    z = model(data.x_dict, data.edge_index_dict)["function"]  # [N, d]
    N = z.size(0)

    # scores for positives
    pos_scores = (z[pos_ei[0]] * z[pos_ei[1]]).sum(dim=-1)

    # build negatives per positive by corrupting destination
    # for each pos (u,v), sample v' negatives
    ranks = []
    hits = 0

    for i in range(pos_ei.size(1)):
        u = pos_ei[0, i]
        v = pos_ei[1, i]

        neg_vs = torch.randint(0, N, (num_neg_per_pos,), device=z.device)
        # scores: positive + negatives
        s_pos = (z[u] * z[v]).sum()
        s_neg = (z[u].unsqueeze(0) * z[neg_vs]).sum(dim=-1)

        # rank of positive among (neg + pos), higher is better
        all_scores = torch.cat([s_neg, s_pos.view(1)], dim=0)
        # descending rank: 1 is best
        rank = (all_scores > s_pos).sum().item() + 1
        ranks.append(rank)
        if rank <= K:
            hits += 1

    mrr = sum(1.0/r for r in ranks) / len(ranks)
    hits_at_k = hits / len(ranks)
    return hits_at_k, mrr

hits10, mrr = eval_hits_mrr(model, data, val_ei, K=15)
# hits10, mrr = eval_hits_mrr(model, data, val_ei, K=10)
print("Hits@10:", hits10, "MRR:", mrr)

In [ ]:
from copy import deepcopy

data_train = deepcopy(data)
data_train[edge_type].edge_index = train_ei


In [ ]:
import torch
from torch_geometric.utils import negative_sampling
from sklearn.metrics import roc_auc_score

@torch.no_grad()
def eval_auc_llm_only(data_train, val_ei, edge_type):
    # Use raw input features directly
    z = data_train['function'].x
    pos = (z[val_ei[0]] * z[val_ei[1]]).sum(dim=-1)

    neg_ei = negative_sampling(val_ei, num_nodes=z.size(0), num_neg_samples=val_ei.size(1))
    neg = (z[neg_ei[0]] * z[neg_ei[1]]).sum(dim=-1)

    y = torch.cat([torch.ones_like(pos), torch.zeros_like(neg)]).cpu().numpy()
    s = torch.cat([pos, neg]).cpu().numpy()
    return roc_auc_score(y, s)

llm_auc = eval_auc_llm_only(data_train, val_ei, edge_type)
print("LLM-only AUC:", llm_auc)


**Next Step clustring**

Constraint-Based Clustering


In [ ]:
import numpy as np
from collections import defaultdict
import random

# ----------------------------
# 0) Helpers
# ----------------------------
def l2_normalize(X, eps=1e-12):
    norms = np.linalg.norm(X, axis=1, keepdims=True)
    return X / (norms + eps)

def safe_edge_fields(e):
    src = e.get("src", e.get("source", e.get("from")))
    dst = e.get("dst", e.get("target", e.get("to")))
    et  = e.get("type", e.get("label", ""))
    w   = e.get("weight", e.get("count", 1))
    return src, dst, et, w

def cosine_sim_pairs(X_norm, pairs):
    # X_norm must be L2-normalized
    return np.array([float(np.dot(X_norm[i], X_norm[j])) for (i, j) in pairs], dtype=np.float32)

In [ ]:
# Expected node schema: {"id": ..., "type": "function", ...}
function_nodes = [n for n in nodes if str(n.get("type", "")).lower() == "function"]
if len(function_nodes) == 0:
    raise ValueError("No function nodes found. Check nodes.json schema: node['type'] must be 'function'.")

func_global_ids = [fn["id"] for fn in function_nodes]
global_to_local_func = {gid: i for i, gid in enumerate(func_global_ids)}

In [ ]:
import json
import os

# 1) Set the path to your embeddings file (adjust if needed)
GNN_EMB_PATH = "/content/gnn_embeddings.json"

if not os.path.exists(GNN_EMB_PATH):
    # try common alternatives
    candidates = [
        "/content/drive/MyDrive/gnn_embeddings.json",
        "/content/gnn_embeddings (1).json",
        "/content/drive/MyDrive/gnn_embeddings (1).json",
    ]
    found = None
    for p in candidates:
        if os.path.exists(p):
            found = p
            break
    if found is None:
        raise FileNotFoundError(
            f"Cannot find gnn_embeddings.json. Checked: {GNN_EMB_PATH} and {candidates}"
        )
    GNN_EMB_PATH = found

with open(GNN_EMB_PATH, "r") as f:
    gnn_embeddings = json.load(f)

print("Loaded gnn_embeddings from:", GNN_EMB_PATH)
print("Type:", type(gnn_embeddings))
if isinstance(gnn_embeddings, dict):
    print("Dict keys sample:", list(gnn_embeddings.keys())[:5])
elif isinstance(gnn_embeddings, list):
    print("List length:", len(gnn_embeddings), "sample item keys:", list(gnn_embeddings[0].keys()) if len(gnn_embeddings) else None)


In [ ]:
def build_X_from_embeddings(emb, func_ids):
    # dict keyed by id
    if isinstance(emb, dict):
        # try both raw key and string key
        def get_vec(gid):
            if gid in emb: return emb[gid]
            if str(gid) in emb: return emb[str(gid)]
            raise KeyError(gid)

        X = np.array([get_vec(gid) for gid in func_ids], dtype=np.float32)
        return X

    # list of objects
    if isinstance(emb, list):
        id_to_vec = {}
        for item in emb:
            # try common fields
            gid = item.get("id", item.get("_id", item.get("node_id")))
            vec = item.get("embedding", item.get("emb", item.get("vector")))
            if gid is not None and vec is not None:
                id_to_vec[gid] = vec
                id_to_vec[str(gid)] = vec
        X = np.array([id_to_vec.get(gid, id_to_vec.get(str(gid))) for gid in func_ids], dtype=np.float32)
        if np.any([v is None for v in X]):
            missing = [gid for gid in func_ids if (gid not in id_to_vec and str(gid) not in id_to_vec)]
            raise KeyError(f"Missing embeddings for {len(missing)} function ids. Example missing: {missing[:5]}")
        return X

    raise TypeError("Unsupported embeddings schema. Must be dict or list.")

X_gnn = np.array([gnn_embeddings[str(i)] for i in range(len(function_nodes))], dtype=np.float32)
if X_gnn.ndim != 2 or X_gnn.shape[0] != len(func_global_ids):
    raise ValueError(f"Bad X_gnn shape: {X_gnn.shape}. Expected (num_functions, dim).")

print("X_gnn:", X_gnn.shape)

# If you do NOT have X_llm in your notebook, do not reference labels_llm anywhere.
HAS_LLM = "X_llm" in globals() and isinstance(globals()["X_llm"], np.ndarray)


In [ ]:
def build_constraints(function_nodes, edges, global_to_local_func, X_norm,
                      tau_must=0.90,
    tau_cannot=0.00,
    max_cannot=2000,
    use_same_file_must=False):
# def build_constraints(function_nodes, edges, global_to_local_func, X_norm,
#                       tau_must=0.85, tau_cannot=0.10,
#                       max_cannot=20000,
#                       use_same_file_must=True):

    must = set()
    cannot = set()

# Xg_norm = l2_normalize(X_gnn)
# must, cannot = build_constraints(
#     function_nodes=function_nodes,
#     edges=train_edges,
#     global_to_local_func=global_to_local_func,
#     X_norm=Xg_norm,
#     tau_must=0.90,
#     tau_cannot=0.00,
#     max_cannot=2000,
#     use_same_file_must=False
# )
    # Must-link from function->function edges (calls/relations)
    for e in edges:
        src, dst, et, w = safe_edge_fields(e)
        if src in global_to_local_func and dst in global_to_local_func:
            i = global_to_local_func[src]
            j = global_to_local_func[dst]
            if i != j:
                must.add((min(i, j), max(i, j)))

    # Optional Must-link from same file (light sampling)
    if use_same_file_must:
        file_to_funcs = defaultdict(list)
        for fn in function_nodes:
            gid = fn["id"]
            if gid in global_to_local_func:
                file_to_funcs[fn.get("file", "")].append(global_to_local_func[gid])

        for f, idxs in file_to_funcs.items():
            if not f or len(idxs) < 2:
                continue
            idxs = sorted(set(idxs))
            # sample adjacent pairs only (prevents explosion)
            for a in range(min(len(idxs) - 1, 50)):
                i, j = idxs[a], idxs[a + 1]
                must.add((min(i, j), max(i, j)))

    # Cannot-link by sampling low-cosine pairs (avoid must pairs)
    n = X_norm.shape[0]
    pairs = set()
    trials = 0
    target_pairs = min(max_cannot * 3, n * n // 50 + 1)

    while len(pairs) < target_pairs and trials < target_pairs * 3:
        i = random.randrange(n)
        j = random.randrange(n)
        if i == j:
            trials += 1
            continue
        a, b = (i, j) if i < j else (j, i)
        if (a, b) in must:
            trials += 1
            continue
        pairs.add((a, b))
        trials += 1

    pairs_list = list(pairs)
    sims = cosine_sim_pairs(X_norm, pairs_list)
    low_idx = np.where(sims <= tau_cannot)[0]

    if len(low_idx) > max_cannot:
        low_idx = np.random.choice(low_idx, size=max_cannot, replace=False)

    for idx in low_idx:
        cannot.add(pairs_list[idx])

    return must, cannot


In [ ]:
def violates_constraints(i, c, labels, must_adj, cannot_adj):
    for j in must_adj[i]:
        lj = labels[j]
        if lj != -1 and lj != c:
            return True
    for j in cannot_adj[i]:
        lj = labels[j]
        if lj != -1 and lj == c:
            return True
    return False

def cop_kmeans(X, k, must, cannot, max_iter=50, seed=42):
    rng = np.random.default_rng(seed)
    n, d = X.shape

    must_adj = defaultdict(set)
    cannot_adj = defaultdict(set)
    for i, j in must:
        must_adj[i].add(j); must_adj[j].add(i)
    for i, j in cannot:
        cannot_adj[i].add(j); cannot_adj[j].add(i)

    centers = X[rng.choice(n, size=k, replace=False)].copy()
    labels = np.full(n, -1, dtype=np.int64)

    for it in range(max_iter):
        changed = False

        order = rng.permutation(n)
        for i in order:
            dists = np.linalg.norm(X[i] - centers, axis=1)
            for c in np.argsort(dists):
                if not violates_constraints(i, c, labels, must_adj, cannot_adj):
                    if labels[i] != c:
                        labels[i] = c
                        changed = True
                    break
            else:
                labels[i] = -1  # no feasible cluster

        new_centers = centers.copy()
        for c in range(k):
            idx = np.where(labels == c)[0]
            if len(idx) > 0:
                new_centers[c] = X[idx].mean(axis=0)
            else:
                new_centers[c] = X[rng.integers(0, n)]
        centers = new_centers

        if not changed:
            break

    return labels

def constraint_violation_rate(labels, must, cannot):
    v = 0
    for i, j in must:
        if labels[i] != -1 and labels[j] != -1 and labels[i] != labels[j]:
            v += 1
    for i, j in cannot:
        if labels[i] != -1 and labels[j] != -1 and labels[i] == labels[j]:
            v += 1
    total = len(must) + len(cannot)
    return v / max(total, 1)


In [ ]:
import numpy as np

def fill_unassigned_by_nearest_center(X, labels, k, seed=42):
    """
    Assigns any label == -1 to the nearest cluster centroid computed
    from currently-assigned points. This is used ONLY to avoid metric
    distortion from dropping edges incident to -1 nodes.
    """
    rng = np.random.default_rng(seed)
    labels_filled = labels.copy()

    # compute centroids from assigned points
    centers = np.zeros((k, X.shape[1]), dtype=np.float32)
    for c in range(k):
        idx = np.where(labels_filled == c)[0]
        if len(idx) > 0:
            centers[c] = X[idx].mean(axis=0)
        else:
            centers[c] = X[rng.integers(0, X.shape[0])]

    # assign unassigned
    un = np.where(labels_filled == -1)[0]
    if len(un) > 0:
        dists = np.linalg.norm(X[un, None, :] - centers[None, :, :], axis=2)
        labels_filled[un] = np.argmin(dists, axis=1)

    return labels_filled


In [ ]:
import numpy as np
from collections import defaultdict
import random

def split_func_edges(edges, global_to_local_func, test_ratio=0.2, seed=42):
    func_edges = []
    for e in edges:
        src, dst, et, w = safe_edge_fields(e)
        if src in global_to_local_func and dst in global_to_local_func:
            func_edges.append(e)

    rng = np.random.default_rng(seed)
    idx = rng.permutation(len(func_edges))
    cut = int(len(func_edges) * (1 - test_ratio))
    train_edges = [func_edges[i] for i in idx[:cut]]
    test_edges  = [func_edges[i] for i in idx[cut:]]
    return train_edges, test_edges, len(func_edges)


def cohesion_coupling_from_edges(labels, edges, global_to_local_func):
    intra = 0
    inter = 0
    used = 0

    for e in edges:
        src, dst, et, w = safe_edge_fields(e)
        if src in global_to_local_func and dst in global_to_local_func:
            i = global_to_local_func[src]
            j = global_to_local_func[dst]
            ci = labels[i]
            cj = labels[j]
            # ignore unassigned
            if ci == -1 or cj == -1:
                continue
            used += 1
            if ci == cj:
                intra += 1
            else:
                inter += 1

    cohesion = intra / max(used, 1)
    coupling = inter / max(used, 1)
    return cohesion, coupling, used, intra, inter

train_edges, test_edges, total_func_edges = split_func_edges(edges, global_to_local_func, test_ratio=0.2, seed=42)
print("total_func_edges:", total_func_edges, "train:", len(train_edges), "test:", len(test_edges))

# Build constraints ONLY from train_edges
Xg_norm = l2_normalize(X_gnn)
must, cannot = build_constraints(
    function_nodes=function_nodes,
    edges=train_edges,
    global_to_local_func=global_to_local_func,
    X_norm=Xg_norm,
    tau_must=0.90,
    tau_cannot=0.00,
    max_cannot=2000,
    use_same_file_must=False
)



# k=15
K=10


labels_gnn = cop_kmeans(X_gnn, k, must, cannot, max_iter=100, seed=42)

# Fill unassigned so evaluation coverage is comparable
labels_gnn = fill_unassigned_by_nearest_center(X_gnn, labels_gnn, k)

# Evaluate ONLY on held-out test_edges
coh_gnn, cou_gnn, used_gnn, intra_gnn, inter_gnn = cohesion_coupling_from_edges(labels_gnn, test_edges, global_to_local_func)
print(f"GNN (HOLDOUT) -> used_edges={used_gnn}, intra={intra_gnn}, inter={inter_gnn}, cohesion={coh_gnn:.4f}, coupling={cou_gnn:.4f}")


# --- LLM Clustering ---
# Load LLM embeddings (assuming they are in node_embeddings.json)
LLM_EMB_PATH = "/content/node_embeddings.json"
with open(LLM_EMB_PATH, "r") as f:
    llm_embeddings = json.load(f)

X_llm = build_X_from_embeddings(llm_embeddings, func_global_ids)
Xl_norm = l2_normalize(X_llm)

# Build constraints for LLM, potentially with different thresholds
must_llm, cannot_llm = build_constraints(
    function_nodes=function_nodes,
    edges=train_edges,
    global_to_local_func=global_to_local_func,
    X_norm=Xl_norm,
    tau_must=0.90,
    tau_cannot=0.00,
    max_cannot=2000,
    use_same_file_must=False
)

labels_llm = cop_kmeans(X_llm, k, must_llm, cannot_llm, max_iter=100, seed=42)
labels_llm = fill_unassigned_by_nearest_center(X_llm, labels_llm, k)


if HAS_LLM:
    coh_llm, cou_llm, used_llm, intra_llm, inter_llm = cohesion_coupling_from_edges(labels_llm, test_edges, global_to_local_func)
    print(f"LLM (HOLDOUT) -> used_edges={used_llm}, intra={intra_llm}, inter={inter_llm}, cohesion={coh_llm:.4f}, coupling={cou_llm:.4f}")


coh_gnn, cou_gnn, used_gnn, intra_gnn, inter_gnn = cohesion_coupling_from_edges(labels_gnn, edges, global_to_local_func)
print(f"GNN clustering -> used_edges={used_gnn}, intra={intra_gnn}, inter={inter_gnn}, cohesion={coh_gnn:.4f}, coupling={cou_gnn:.4f}")

if HAS_LLM:
    coh_llm, cou_llm, used_llm, intra_llm, inter_llm = cohesion_coupling_from_edges(labels_llm, edges, global_to_local_func)
    print(f"LLM clustering -> used_edges={used_llm}, intra={intra_llm}, inter={inter_llm}, cohesion={coh_llm:.4f}, coupling={cou_llm:.4f}")

In [ ]:
def clusters_as_dict(labels, function_nodes):
    clusters = defaultdict(list)
    for idx, c in enumerate(labels):
        if c == -1:
            continue
        clusters[int(c)].append(function_nodes[idx])
    return dict(clusters)

clusters_gnn = clusters_as_dict(labels_gnn, function_nodes)
print("num_clusters_nonempty:", len(clusters_gnn))
print("sizes:", sorted([len(v) for v in clusters_gnn.values()], reverse=True)[:8])

Time To Get OutPut

In [ ]:
from collections import defaultdict

def build_cluster_groups(labels, function_nodes):
    groups = defaultdict(list)
    for i, c in enumerate(labels):
        groups[int(c)].append(function_nodes[i])
    return dict(groups)

clusters_gnn = build_cluster_groups(labels_gnn, function_nodes)

print("Non-empty clusters:", len(clusters_gnn))
print("Cluster sizes:", sorted([len(v) for v in clusters_gnn.values()], reverse=True))


In [ ]:
def node_name(n):
    # adjust if your node schema uses different fields
    return n.get("name") or n.get("qualified_name") or n.get("signature") or str(n.get("id"))

for cid in sorted(clusters_gnn.keys()):
    items = clusters_gnn[cid]
    print(f"\n=== Microservice {cid} | size={len(items)} ===")
    for n in items[:15]:
        print(" -", node_name(n))


In [ ]:
import json

func_to_cluster = []
for i, fn in enumerate(function_nodes):
    func_to_cluster.append({
        "id": fn.get("id"),
        "name": node_name(fn),
        "file": fn.get("file"),
        "cluster": int(labels_gnn[i]),
    })

with open("/content/gnn_clusters_function_map.json", "w") as f:
    json.dump(func_to_cluster, f, indent=2)

print("Saved:", "/content/gnn_clusters_function_map.json")


In [ ]:
cluster_to_funcs = {}
for cid, items in clusters_gnn.items():
    cluster_to_funcs[str(cid)] = [{
        "id": it.get("id"),
        "name": node_name(it),
        "file": it.get("file"),
    } for it in items]

with open("/content/gnn_clusters_grouped.json", "w") as f:
    json.dump(cluster_to_funcs, f, indent=2)

print("Saved:", "/content/gnn_clusters_grouped.json")


In [ ]:
import csv

with open("/content/gnn_clusters.csv", "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f)
    w.writerow(["id", "name", "file", "cluster"])
    for row in func_to_cluster:
        w.writerow([row["id"], row["name"], row["file"], row["cluster"]])

print("Saved:", "/content/gnn_clusters.csv")


Finding K


In [ ]:
k_values = range(4, 30)

results = []

for k in range(8, 30):
    # --- clustering ---
    labels = cop_kmeans(X_gnn, k, must, cannot, max_iter=100, seed=42)
    labels = fill_unassigned_by_nearest_center(X_gnn, labels, k)

    # --- evaluation on HELD-OUT edges ---
    coh, cou, used, intra, inter = cohesion_coupling_from_edges(
        labels, test_edges, global_to_local_func
    )

    score = coh - cou   # λ = 1

    cluster_sizes = [np.sum(labels == i) for i in range(k)]

    results.append({
        "k": k,
        "cohesion": coh,
        "coupling": cou,
        "score": score,
        "min_size": min(cluster_sizes),
        "max_size": max(cluster_sizes),
    })



In [ ]:
best_score = max(r["score"] for r in results)

candidates = [
    r for r in results
    if r["score"] >= 0.95 * best_score
    and r["min_size"] >= 5
    and r["max_size"] <= 0.5 * len(function_nodes)
]

best_k = min(r["k"] for r in candidates)
print("Best k:", best_k)


In [ ]:
import numpy as np
from sklearn.metrics import silhouette_score

# --- Edge helpers (use your existing safe_edge_fields/global_to_local_func) ---
def cohesion_coupling_from_edges(labels, edges, global_to_local_func):
    intra = inter = used = 0
    dropped_unassigned = 0
    total_func_edges = 0

    for e in edges:
        src, dst, et, w = safe_edge_fields(e)
        if src in global_to_local_func and dst in global_to_local_func:
            total_func_edges += 1
            i = global_to_local_func[src]
            j = global_to_local_func[dst]
            ci, cj = labels[i], labels[j]

            if ci == -1 or cj == -1:
                dropped_unassigned += 1
                continue

            used += 1
            if ci == cj:
                intra += 1
            else:
                inter += 1

    cohesion = intra / max(used, 1)
    coupling = inter / max(used, 1)
    coverage = used / max(total_func_edges, 1)
    return {
        "total_func_edges": total_func_edges,
        "used_edges": used,
        "dropped_unassigned": dropped_unassigned,
        "coverage": coverage,
        "intra": intra,
        "inter": inter,
        "cohesion": cohesion,
        "coupling": coupling,
    }

def silhouette_safe(X, labels):
    labels = np.asarray(labels)
    mask = labels != -1
    X2 = X[mask]
    y2 = labels[mask]

    # silhouette needs at least 2 clusters and no empty/degenerate cases
    uniq = np.unique(y2)
    if len(uniq) < 2 or len(y2) < 3:
        return None
    # also fails if any cluster has size 1 (can be allowed but may be unstable)
    counts = {u: int(np.sum(y2 == u)) for u in uniq}
    if min(counts.values()) < 2:
        # still can compute, but often noisy; return anyway if you want:
        try:
            return float(silhouette_score(X2, y2, metric="euclidean"))
        except Exception:
            return None

    try:
        return float(silhouette_score(X2, y2, metric="euclidean"))
    except Exception:
        return None

# -----------------------
# RUN METRICS (FULL EDGES)
# -----------------------
metrics_full = cohesion_coupling_from_edges(labels_gnn, edges, global_to_local_func)
sil_full = silhouette_safe(X_gnn, labels_gnn)

print("=== GNN clustering (FULL) ===")
print(f"coverage={metrics_full['coverage']:.3f}  used_edges={metrics_full['used_edges']}/{metrics_full['total_func_edges']}  dropped_unassigned={metrics_full['dropped_unassigned']}")
print(f"intra={metrics_full['intra']}  inter={metrics_full['inter']}")
print(f"cohesion={metrics_full['cohesion']:.4f}  coupling={metrics_full['coupling']:.4f}")
print("silhouette:", "N/A" if sil_full is None else f"{sil_full:.4f}")

# --------------------------
# OPTIONAL: HOLDOUT METRICS
# --------------------------
# If you have test_edges from your split, you can evaluate on holdout too:
if "test_edges" in globals():
    metrics_test = cohesion_coupling_from_edges(labels_gnn, test_edges, global_to_local_func)
    sil_test = silhouette_safe(X_gnn, labels_gnn)  # silhouette is geometry-only; no "holdout"

    print("\n=== GNN clustering (HOLDOUT edges) ===")
    print(f"coverage={metrics_test['coverage']:.3f}  used_edges={metrics_test['used_edges']}/{metrics_test['total_func_edges']}  dropped_unassigned={metrics_test['dropped_unassigned']}")
    print(f"intra={metrics_test['intra']}  inter={metrics_test['inter']}")
    print(f"cohesion={metrics_test['cohesion']:.4f}  coupling={metrics_test['coupling']:.4f}")
    print("silhouette (same as full):", "N/A" if sil_test is None else f"{sil_test:.4f}")

# --------------------------
# OPTIONAL: LLM COMPARISON
# --------------------------
if "labels_llm" in globals() and "X_llm" in globals():
    m_llm = cohesion_coupling_from_edges(labels_llm, edges, global_to_local_func)
    sil_llm = silhouette_safe(X_llm, labels_llm)

    print("\n=== LLM clustering (FULL) ===")
    print(f"coverage={m_llm['coverage']:.3f}  used_edges={m_llm['used_edges']}/{m_llm['total_func_edges']}  dropped_unassigned={m_llm['dropped_unassigned']}")
    print(f"intra={m_llm['intra']}  inter={m_llm['inter']}")
    print(f"cohesion={m_llm['cohesion']:.4f}  coupling={m_llm['coupling']:.4f}")
    print("silhouette:", "N/A" if sil_llm is None else f"{sil_llm:.4f}")
    #K=10


Compared to **GNN (FULL)** vs **LLM (FULL)**:

1. **Cohesion** increased from `0.4721` to `0.6034`
   → **+0.1313** (about **+13.13 percentage points**).
   In simple terms: a larger share of edges ended up **within** clusters.

2. **Coupling** decreased from `0.5279` to `0.3966`
   → **−0.1313** (about **−13.13 percentage points**, which is better).
   Meaning: fewer edges were **between** clusters.

3. **Silhouette** increased from `0.0510` to `0.0951`
   → **+0.0441** (about **1.86×** higher).


In [ ]:
from pyvis.network import Network
import random

# ----------------------------
# 0) Color palette (stable, distinct)
# ----------------------------
PALETTE = [
    "#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd",
    "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22", "#17becf",
    "#393b79", "#637939", "#8c6d31", "#843c39", "#7b4173"
]

def color_for_cluster(c):
    if c is None or c == -1:
        return "#c7c7c7"  # unassigned/unknown
    return PALETTE[int(c) % len(PALETTE)]

def node_display_name(n):
    return n.get("name") or n.get("qualified_name") or n.get("signature") or str(n.get("id"))

def safe_edge_fields(e):
    src = e.get("src", e.get("source", e.get("from")))
    dst = e.get("dst", e.get("target", e.get("to")))
    et  = e.get("type", e.get("label", ""))
    w   = e.get("weight", e.get("count", 1))
    return src, dst, et, w

# ----------------------------
# 1) Build node_id -> cluster map (for function nodes)
# ----------------------------
func_id_to_cluster = {}
for i, fn in enumerate(function_nodes):
    func_id_to_cluster[fn["id"]] = int(labels_gnn[i])

# ----------------------------
# 2) Create network
# ----------------------------
net = Network(height="800px", width="100%", bgcolor="#ffffff", font_color="#111111", directed=True)
net.barnes_hut(gravity=-8000, central_gravity=0.3, spring_length=120, spring_strength=0.01, damping=0.09)

# ----------------------------
# 3) Add nodes with cluster colors
# ----------------------------
for n in nodes:
    nid = n.get("id")
    ntype = str(n.get("type", ""))
    c = func_id_to_cluster.get(nid, None) if ntype.lower() == "function" else None

    title_parts = [
        f"Type: {ntype}",
        f"ID: {nid}",
    ]
    if c is not None:
        title_parts.append(f"Cluster: {c}")

    # size: make function nodes slightly larger
    size = 18 if ntype.lower() == "function" else 10

    net.add_node(
        nid,
        label=node_display_name(n),
        title="<br>".join(title_parts),
        color=color_for_cluster(c) if ntype.lower() == "function" else "#e0e0e0",
        size=size,
    )

# ----------------------------
# 4) Add edges
# ----------------------------
for e in edges:
    src, dst, et, w = safe_edge_fields(e)
    if src is None or dst is None:
        continue

    # optional: edge width by weight (kept bounded)
    try:
        width = max(1, min(6, float(w)))
    except Exception:
        width = 1

    net.add_edge(
        src,
        dst,
        title=str(et),
        width=width
    )

# ----------------------------
# 5) Legend (clusters) as dummy nodes (optional but useful)
# ----------------------------
# Add a small legend column on the left
legend_x = -800
legend_y = -400
for c in sorted(set(func_id_to_cluster.values())):
    net.add_node(
        f"legend_cluster_{c}",
        label=f"Cluster {c}",
        color=color_for_cluster(c),
        shape="box",
        x=legend_x,
        y=legend_y + 60 * c,
        fixed=True,
        physics=False,
        size=14,
    )

# ----------------------------
# 6) Save HTML
# ----------------------------
out_path = "/content/graph_clustered.html"

# IMPORTANT: write_html avoids some notebook/template issues that show() triggers
net.write_html(out_path, open_browser=False)
print("Saved clustered graph to:", out_path)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

def find_optimal_k(embeddings, max_k=20):
    scores = []
    k_values = range(2, max_k + 1)

    for k in k_values:
        # شبیه‌سازی کلاستری که در مدل GNN استفاده کردید
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        labels = kmeans.fit_predict(embeddings)
        score = silhouette_score(embeddings, labels)
        scores.append(score)
        print(f"K={k}, Silhouette Score={score:.4f}")

    plt.figure(figsize=(10, 6))
    plt.plot(k_values, scores, 'bo-')
    plt.xlabel('Number of Clusters (K)')
    plt.ylabel('Silhouette Score')
    plt.title('Optimal K Selection (Elbow Method)')
    plt.axvline(x=10, color='r', linestyle='--', label='Selected K')
    plt.legend()
    plt.show()

# اجرا روی ویژگی‌های استخراج شده
find_optimal_k(X_gnn)
